In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib

In [ ]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')


In [ ]:
import os
os.getcwd()

In [ ]:
data_dir = 'D:\Brain Cancer\Dataset\Brain_Cancer_processed_4'
os.listdir(data_dir)

In [ ]:
tf.random.set_seed(42)

train_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'training', seed = 1, shuffle = True, batch_size = 32, image_size=(256,256))

test_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'validation', seed = 1, shuffle = True, batch_size = 32, image_size=(256,256))

In [ ]:
filenames = pathlib.Path(data_dir)
for label in train_data.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_data.cardinality().numpy(),  test_data.cardinality().numpy()

In [ ]:
train_set = train_data.take(152)
val_set = train_data.skip(152)

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy()

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
X_test, y_test = None, None
for images, labels in test_data:
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
from tensorflow.keras.applications import VGG19

In [ ]:
VGG19Model = VGG19(include_top = False, weights = 'imagenet', input_shape = (256, 256, 3))

In [ ]:
VGG19Model.summary()

In [ ]:
VGG19Model.trainable = True

In [ ]:
from tensorflow.keras.layers import Rescaling, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model

In [ ]:
tf.random.set_seed(42)

X = Rescaling(1./255)
X = VGG19Model.output
X = GlobalAveragePooling2D()(X)
X = Dense(1024, activation='relu')(X)
X = Dropout(0.5)(X)
X = Dense(512, activation='relu')(X)
X = Dropout(0.25)(X)
predictions = Dense(3, activation='softmax')(X)

In [ ]:
model2=Model(inputs=VGG19Model.input,outputs=predictions)

In [ ]:
model2.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

In [ ]:
history_2 = model2.fit(train_set, epochs=20, validation_data=val_set)

In [ ]:
model2.summary()

In [ ]:
# Save the model
#model2.save('d:\\Balanced Augmented Covid CXR Dataset\\Model\\VGG19Model.h5')

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
        # Specify the directory to save the PDF
    save_dir = 'D:\\Brain Cancer\\PDF'
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Full path to save the PDF file
    pdf_path = os.path.join(save_dir, 'VGG19_training_curves.pdf')
    
    # Save the plot as a PDF with tight layout
    plt.tight_layout()  # Ensure that layout is tight
    plt.savefig(pdf_path, format='pdf', bbox_inches='tight')  
    # Show the plot
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_2.history))

In [ ]:
from sklearn import metrics

In [ ]:
y_pred_proba = model2.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
test_score = model2.evaluate(test_data, verbose= 1)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['brain_glioma', 'brain_menin', 'brain_tumor']
print(classification_report(y_test , y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
plt.figure(figsize = (6,6), dpi = 100)
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, fmt='d', cmap = 'Greens')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
    # Add a title with the model name VGG19
plt.title('Confusion Matrix for VGG19')

    # Specify the directory to save the PDF
save_dir = 'D:\\Brain Cancer\\PDF'
    
    # Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

    # Full path to save the PDF file (with VGG19 in the filename)
pdf_path = os.path.join(save_dir, 'VGG19_confusion_matrix.pdf')
    
    # Save the plot as a PDF with tight layout
plt.tight_layout()  # Ensure that layout is tight
plt.savefig(pdf_path, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)

In [ ]:
plot_random_predictions(test_data, model2)